In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# training and evaluation

In [ ]:
! pip install Pillow-SIMD
! pip install facenet-pytorch
! pip install ffmpeg-python


In [1]:
import sys
sys.path.insert(0,'/content/drive/My Drive/my_projects/ISLR/Real-time-GesRec-master')
sys.path.insert(1,'/content/drive/My Drive/my_projects/ISLR')

In [2]:
from pathlib import Path

no_val=False#True
testset=False

path_det=Path('/content/drive/My Drive/my_projects/ISLR/pretrained_models/egogesture_resnetl_10_RGB_8.pth')
path_clf=Path('/content/drive/My Drive/my_projects/ISLR/my_model/n/with_none/merged_testset/best_acc_model_state_dict.pth')
#pretrained_path_clf=Path('/content/drive/My Drive/my_projects/ISLR/model_checkpoint/checkpoint.pth')
pretrained_path_clf=Path('/content/drive/My Drive/my_projects/ISLR/pretrained_models/jester_resnext_101_RGB_16_best.pth')
#ann_path=Path('/content/drive/My Drive/my_projects/ISLR/annotation/annotation_0-9_w_nf_w_none.json')
ann_path=Path('/content/drive/My Drive/my_projects/ISLR/annotation/annotation_0-9_w_nf_w_none_mrg_testset.json')

#vid_path='E:\\Faiz\\Projects\\My_Projects\\ISLR\\3DCNN\\my_video\\ego.mp4'
vid_path=''#'E:\\Faiz\\Projects\\My_Projects\\ISLR'
res_path=Path('/content/drive/My Drive/my_projects/ISLR/my_result')
root_path='/content/drive/My Drive/my_projects/ISLR'
classifier='resnext'

In [3]:

class Opt():
    def __init__(self):
        self.root_path = root_path
        self.resume_path = ''  # path_jstr_clf
        self.sample_duration = 16
        self.pretrain_path = pretrained_path_clf
        self.is_pretrained_model_has_dropout=False           ##### only my models has dropout 
        self.n_classes = 27
        self.n_finetune_classes = 11#10

        self.model = classifier
        self.model_depth = 101
        self.width_mult = 1.0
        self.modality = 'RGB'
        self.resnet_shortcut = 'B'
        self.arch = ''
        self.scales = ''
        self.initial_scale = 1.0
        self.n_scales = 1
        self.scale_step = 0.84089641525
        self.ft_begin_index = 0
        self.ft_portion = 'last_layer'
        self.mean = [114.7748, 107.7354, 99.475]
        self.std = [38.7568578, 37.88248729, 40.02898126]
        self.resnext_cardinality = 32
        self.pretrain_modality = 'RGB'
        self.no_cuda = False
        self.norm_value = 1
        self.no_mean_norm = False
        self.std_norm = False
        self.sample_size = 112
        self.manual_seed = 1
        self.stride_len = 1
        self.video_path = vid_path
        self.annotation_path = ann_path
        self.result_path = res_path
        self.store_name = 'model'
        self.dataset = 'egogesture'
        self.no_train = False
        self.topk=1
        self.train_crop = 'center'
        self.downsample = 1               #### used in temporal transformation
        self.batch_size = 4
        self.n_threads = 0
        self.nesterov = True
        self.dampening = 0.9
        self.learning_rate = .01
        self.lr_steps = [150]#[8,16,20,23]#[10, 20, 30, 40, 100]
        self.momentum = 0.9
        self.weight_decay = 1e-3
        self.lr_patience = 10
        self.no_val = no_val
        self.n_val_samples=1
        self.val_batch_size=1#4
        self.begin_epoch = 1
        self.n_epochs = 1
        self.test=testset
        self.test_subset='test'
        self.test_batch_size=1#4
        #        self.scale_in_test=''
        #        self.crop_position_in_test=''
        
        self.mean_dataset = 'activitynet'
        self.with_egogesture = False
        self.train_validate = False
        self.feature_extracting=True

        self.resnext_dropout=True
        self.resnext_dropout_prob=.5


        self.show_1_batch=False
        self.figsize=(6,6)



In [4]:
resume=False
resume_path='/content/drive/My Drive/my_projects/ISLR/model_checkpoint/'+'with_none/wo_val_test/'+'checkpoint.pth'

if resume:
  opening_mode='a'
else: opening_mode='w'

In [5]:
import os
import sys
import json
import numpy as np
import torch
from torch import nn
from torch import optim
from torch.optim import lr_scheduler

# from opts import parse_opts
from model import generate_model
from mean import get_mean, get_std
from spatial_transforms import *
from temporal_transforms import *
from target_transforms import ClassLabel, VideoID
from target_transforms import Compose as TargetCompose
from dataset import get_training_set, get_validation_set, get_test_set
from utils import *
from train import train_epoch
from validation import val_epoch
import test

#from my_input import *

best_prec1 = 0

if __name__ == '__main__':
#    opt = parse_opts()
    opt = Opt()
    if opt.root_path != '':
        opt.video_path = os.path.join(opt.root_path, opt.video_path)
        opt.annotation_path = os.path.join(opt.root_path, opt.annotation_path)
        opt.result_path = os.path.join(opt.root_path, opt.result_path)
        if not os.path.exists(opt.result_path):
            os.makedirs(opt.result_path)
        if opt.resume_path:
            opt.resume_path = os.path.join(opt.root_path, opt.resume_path)
        if opt.pretrain_path:
            opt.pretrain_path = os.path.join(opt.root_path, opt.pretrain_path)
    opt.scales = [opt.initial_scale]
    for i in range(1, opt.n_scales):
        opt.scales.append(opt.scales[-1] * opt.scale_step)
    opt.arch = '{}'.format(opt.model)
    opt.mean = get_mean(opt.norm_value, dataset=opt.mean_dataset)
    opt.std = get_std(opt.norm_value)
    opt.store_name = '_'.join([opt.dataset, opt.model, str(opt.width_mult) + 'x',
                               opt.modality, str(opt.sample_duration)])
    
#    with open(os.path.join(opt.result_path, 'opts.json'), 'w') as opt_file:
#        json.dump(vars(opt), opt_file)

    torch.manual_seed(opt.manual_seed)
    model=generate_model(opt)
    if isinstance(model,tuple):
       model, parameters = model
                        
#    for param in model.parameters():
#        print(param.requires_grad) 

#    print(model)

    # Egogesture, with "no-gesture" training, weighted loss
    # class_weights = torch.cat((0.012*torch.ones([1, 83]), 0.00015*torch.ones([1, 1])), 1)
    criterion = nn.CrossEntropyLoss()

    # # nvgesture, with "no-gesture" training, weighted loss
    # class_weights = torch.cat((0.04*torch.ones([1, 25]), 0.0008*torch.ones([1, 1])), 1)
    # criterion = nn.CrossEntropyLoss(weight=class_weights, size_average=False)

    # criterion = nn.CrossEntropyLoss()
    if not opt.no_cuda:
        model=model.cuda()
        criterion = criterion.cuda()

    if opt.no_mean_norm and not opt.std_norm:
        norm_method = Normalize([0, 0, 0], [1, 1, 1])
    elif not opt.std_norm:
        norm_method = Normalize(opt.mean, [1, 1, 1])
    else:
        norm_method = Normalize(opt.mean, opt.std)

    if not opt.no_train:
        assert opt.train_crop in ['random', 'corner', 'center']
        if opt.train_crop == 'random':
            crop_method = MultiScaleRandomCrop(opt.scales, opt.sample_size)
        elif opt.train_crop == 'corner':
            crop_method = MultiScaleCornerCrop(opt.scales, opt.sample_size)
        elif opt.train_crop == 'center':
            crop_method = MultiScaleCornerCrop(
                opt.scales, opt.sample_size, crop_positions=['c'])
        spatial_transform = Compose([
            #RandomHorizontalFlip(),
            RandomRotate(),
            #RandomResize(),
            #crop_method,
            ExtractRoi(),
            RandomResize(),           ####
            FinalSize(),              ####
            RandomBrightnessContrast(),  ####
            Random_Perspective(),       ####
            #MultiplyValues(),
            #Dropout(),
            #SaltImage(),
            #Gaussian_blur(),
            SpatialElasticDisplacement(),
            ToTensor(opt.norm_value), norm_method
        ])
        temporal_transform = TemporalRandomCrop(opt.sample_duration, opt.downsample)
        target_transform = ClassLabel()
        training_data = get_training_set(opt, spatial_transform,
                                         temporal_transform, target_transform)
        train_loader = torch.utils.data.DataLoader(
            training_data,
            batch_size=opt.batch_size,
            shuffle=True,
            num_workers=opt.n_threads,
            pin_memory=True)
        train_logger = Logger(
            os.path.join(opt.result_path, opt.store_name + '_train.log'),
            ['epoch', 'loss', 'prec1', 'lr'],opening_mode)   #####
        train_batch_logger = Logger(
            os.path.join(opt.result_path, 'train_batch.log'),
            ['epoch', 'batch', 'iter', 'loss', 'prec1', 'lr'],opening_mode)

        if opt.nesterov:
            dampening = 0
        else:
            dampening = opt.dampening
        optimizer = optim.SGD(
            parameters,
            lr=opt.learning_rate,
            momentum=opt.momentum,
            dampening=dampening,
            weight_decay=opt.weight_decay,
            nesterov=opt.nesterov)
        scheduler = lr_scheduler.ReduceLROnPlateau(
            optimizer, 'min', patience=opt.lr_patience)
    if not opt.no_val:
        spatial_transform = Compose([
          #  Scale(opt.sample_size),    #####
            #CenterCrop(opt.sample_size),  ##### 
            ExtractRoi(),
            FinalSize(),        
            ToTensor(opt.norm_value), norm_method
        ])
        #temporal_transform = LoopPadding(opt.sample_duration)
        temporal_transform = TemporalCenterCrop(opt.sample_duration, opt.downsample)
        target_transform = ClassLabel()
        validation_data = get_validation_set(
            opt, spatial_transform, temporal_transform, target_transform)
        val_loader = torch.utils.data.DataLoader(
            validation_data,
            batch_size=opt.val_batch_size,
            shuffle=False,
            num_workers=opt.n_threads,
            pin_memory=True)
        val_logger = Logger(
            os.path.join(opt.result_path, opt.store_name + '_val.log'), ['epoch', 'loss', 'prec1'],opening_mode)

    if opt.test:
        spatial_transform = Compose([
          #  Scale(opt.sample_size),    #####
            #CenterCrop(opt.sample_size),  #####  
            ExtractRoi(),
            FinalSize(),           
            ToTensor(opt.norm_value), norm_method
        ])
        #temporal_transform = LoopPadding(opt.sample_duration)
        temporal_transform = TemporalCenterCrop(opt.sample_duration, opt.downsample)
        target_transform = ClassLabel()
        test_data = get_test_set(
            opt, spatial_transform, temporal_transform, target_transform)
        test_loader = torch.utils.data.DataLoader(
            test_data,
            batch_size=opt.test_batch_size,
            shuffle=False,
            num_workers=opt.n_threads,
            pin_memory=True)
        test_logger = Logger(
            os.path.join(opt.result_path, opt.store_name + '_test.log'), ['epoch', 'loss', 'prec1'],opening_mode)


    if opt.resume_path:
        print('loading checkpoint {}'.format(opt.resume_path))
        checkpoint = torch.load(opt.resume_path)
        assert opt.arch == checkpoint['arch']
        best_prec1 = checkpoint['best_prec1']
        opt.begin_epoch = checkpoint['epoch']
        model.load_state_dict(checkpoint['state_dict'])



Total number of trainable parameters:  0
loading pretrained model /content/drive/My Drive/my_projects/ISLR/pretrained_models/jester_resnext_101_RGB_16_best.pth
[INFO]: EgoGesture Dataset - training, is loading...
dataset loading [0/619]
[INFO]: EgoGesture Dataset - validation, is loading...
dataset loading [0/107]


In [14]:

if resume:
    checkpoint = torch.load(resume_path)
    assert opt.arch == checkpoint['arch']
    model.load_state_dict(checkpoint['state_dict'])
    opt.begin_epoch = checkpoint['epoch']+1
    train_best_prec1 = checkpoint['train_best_prec1']
    train_best_loss = checkpoint['train_best_loss']
    if not opt.no_val:
        best_prec1 = checkpoint['best_prec1']
        best_loss = checkpoint['best_loss']
    if opt.test:
        test_best_prec1 = checkpoint['test_best_prec1']
        test_best_loss = checkpoint['test_best_loss']
else:
    train_best_loss=10
    best_loss=10
    test_best_loss=10
    train_best_prec1=0
    best_prec1 = 0
    test_best_prec1=0

train_loss=[]
val_loss=[]
test_loss=[]
train_acc=[]
val_acc=[]
test_acc=[]

def fit(lr,epochs,model_checkpoint_path):
    global train_best_prec1,best_prec1,test_best_prec1,train_best_loss,best_loss,test_best_loss
    global train_loss,val_loss,test_loss,train_acc,val_acc,test_acc

    opt.learning_rate=lr
    opt.n_epochs=epochs
    
    print('run')
    for i in range(opt.begin_epoch, opt.n_epochs + 1):
    # for i in range(opt.begin_epoch, opt.begin_epoch + 10):
        if not opt.no_train:
            adjust_learning_rate(optimizer, i, opt)
 #            print('lr first and last layer\n',optimizer.param_groups[0]['lr'],optimizer.param_groups[313]['lr'])
            loss,acc=train_epoch(i, train_loader, model, criterion, optimizer, opt,
                        train_logger, train_batch_logger)
            
            train_loss.append(loss)
            train_acc.append(acc) 

            if acc > train_best_prec1:
               train_best_prec1=acc
               torch.save(model.state_dict(),'/content/drive/My Drive/my_projects/ISLR/my_model/n/with_none/wo_val_test/train_best_acc_model_state_dict.pth')

            if loss<train_best_loss:
                train_best_loss=loss
                torch.save(model.state_dict(),'/content/drive/My Drive/my_projects/ISLR/my_model/n/with_none/wo_val_test/train_best_loss_model_state_dict.pth')
   

            state = {
                'epoch': i,
                'arch': opt.arch,
                'state_dict': model.state_dict(),
                'optimizer': optimizer.state_dict(),
                'train_best_prec1': train_best_prec1,
                'train_best_loss': train_best_loss
                }
            if not opt.no_val:
              state.update({'best_prec1': best_prec1,
                            'best_loss': best_loss})
            if opt.test:
              state.update({'test_best_prec1': test_best_prec1,
                            'test_best_loss': test_best_loss})
           # save_checkpoint(state, False, opt)  ####

            torch.save(state,model_checkpoint_path)
            
        if not opt.no_val:
            print('validation at epoch {}'.format(i))
            validation_loss, prec1 = val_epoch(i, val_loader, model, criterion, opt,
                                        val_logger)
            
            val_loss.append(validation_loss)    ####
            val_acc.append(prec1)

            is_best = prec1 > best_prec1
            best_prec1 = max(prec1, best_prec1)
            if is_best:
               torch.save(model.state_dict(),'/content/drive/My Drive/my_projects/ISLR/my_model/n/with_none/merged_testset/best_acc_model_state_dict.pth')

            if validation_loss<best_loss:
                best_loss=validation_loss
                torch.save(model.state_dict(),'/content/drive/My Drive/my_projects/ISLR/my_model/n/with_none/merged_testset/best_loss_model_state_dict.pth')
                
            state = {
                'epoch': i,
                'arch': opt.arch,
                'state_dict': model.state_dict(),
                'optimizer': optimizer.state_dict(),
                'best_prec1': best_prec1
                }
           # save_checkpoint(state, is_best, opt)         ####

        if opt.test:
            print('testing at epoch {}'.format(i))
            testing_loss, test_prec1 = val_epoch(i, test_loader, model, criterion, opt,
                                        test_logger)
            
            test_loss.append(testing_loss)    ####
            test_acc.append(test_prec1)

            test_is_best = test_prec1 > test_best_prec1
            test_best_prec1 = max(test_prec1, test_best_prec1)
            if test_is_best:
               torch.save(model.state_dict(),'/content/drive/My Drive/my_projects/ISLR/my_model/n/with_none/merged_testset/test_best_acc_model_state_dict.pth')

            if testing_loss<test_best_loss:
                test_best_loss=testing_loss
                torch.save(model.state_dict(),'/content/drive/My Drive/my_projects/ISLR/my_model/n/with_none/merged_testset/test_best_loss_model_state_dict.pth')
                
            state = {
                'epoch': i,
                'arch': opt.arch,
                'state_dict': model.state_dict(),
                'optimizer': optimizer.state_dict(),
                'best_prec1': test_best_prec1
                }

        
        print('train best prec:',train_best_prec1,'train best loss:',train_best_loss)
        if not opt.no_val:
            print('val best prec:',best_prec1,'val best loss:',best_loss)
        if opt.test:
            print('test best prec:',test_best_prec1,'test best loss:',test_best_loss)
        print('\n')

In [15]:
print('train best prec:',train_best_prec1,'train best loss:',train_best_loss)#'\nval best prec:',best_prec1,'val best loss:',best_loss,'\ntest best prec:',test_best_prec1,'test best loss:',test_best_loss,'\n')

train best prec: 0 train best loss: 10


In [16]:

opt.begin_epoch

1

In [18]:
epochs=125
model_checkpoint_name='tmp.pth'#'checkpoint.pth'
model_checkpoint_path='/content/drive/My Drive/my_projects/ISLR/model_checkpoint/'+'with_none/merged_testset/'+model_checkpoint_name

In [ ]:
## training last layer
for param in model.parameters():
      print(param.requires_grad)

In [ ]:
lr=0.001
fit(lr,epochs,model_checkpoint_path)

In [ ]:
## fine tuning the model
for param in model.parameters():
      param.requires_grad=True

In [ ]:
lr=0.0001
fit(lr,epochs,model_checkpoint_path)

## plot top loss

In [11]:
import torch.nn.functional as F
import matplotlib.pyplot as plt
import torch
from torch.autograd import Variable
import numpy as np
import cv2

def plot_top_losses(topk=2,model=model,data_loader=None):
    preds=torch.tensor([]).cuda()
    targs=torch.tensor([]).cuda()

    model.eval()
    j=1
    for i,t in data_loader:

      t=t.cuda()
     # with torch.no_grad():
     #     inputs=Variable(i) 
     #     outputs=Variable(t)
      inputs=i
      outputs=t
      pred=model(inputs)
      preds=torch.cat((preds.data,pred),0)
      targs=torch.cat((targs,outputs),0)


    samp_loss=F.cross_entropy(preds,targs.long(),reduction='none')
    assert len(samp_loss)==len(data_loader.dataset.data)
    
    pred_cls=preds.argmax(dim=1)

    print('accuracy:',sum(torch.eq(pred_cls,targs))*100/len(targs))

    loss_indx=samp_loss.argsort(descending=True)
    path=[]
    segment=[]
    for k,i in enumerate(loss_indx[:topk]):
        i=int(i.cpu())
        path.append(data_loader.dataset.data[i]['video'])
        segment.append(data_loader.dataset.data[i]['segment'])
        print('\n\n',k,'\npath:',path[-1],'\nsegment',segment[-1],'\nloss:',samp_loss[i].data,'\nprediction',F.softmax(preds[i].data),'\npred_cls:',pred_cls[i],',\tact_cls:',targs[i])
        frm_arr=data_loader.dataset[i][0].permute(1,2,3,0)
        frm_arr=(frm_arr+torch.tensor(opt.mean)).int()
        rows,cols=2,8
        ln=16
        fig,ax=plt.subplots(rows,cols,figsize=(cols*2.25,rows*2.5))
        for i in range(ln):
            frame=cv2.cvtColor(np.uint8(frm_arr[i]),cv2.COLOR_BGR2RGB)
            ax[i//cols,i%cols].imshow(frame)
            ax[i//cols,i%cols].set_title(i)
        plt.show()

In [7]:
import gc
gc.collect()

2009

In [8]:
#torch.cuda.empty_cache()
torch.cuda.memory_reserved()/10**9

0.201326592

In [9]:
#model=torch.load('/content/drive/MyDrive/my_projects/ISLR/my_model/classifier_whole_model.pth')
model.load_state_dict(torch.load('/content/drive/MyDrive/my_projects/ISLR/my_model/n/with_none/merged_testset/best_acc_model_state_dict.pth'))

<All keys matched successfully>

In [ ]:
# with none_class and more_epoch model
plot_top_losses(20,model, val_loader)
torch.cuda.empty_cache()
torch.cuda.memory_reserved()

In [ ]:
# with none_class and more_epoch model
plot_top_losses(80,model, test_loader)
torch.cuda.empty_cache()
torch.cuda.memory_reserved()